In [3]:
####eyeML
# 1. 얼굴 영역 검출
# 2. 얼굴 영역 속 눈 영역 검출
# 3. 왼쪽, 오른쪽 둘다 검출
# 4. 카메라 이미지 전처리
# 5. 모델에 적용 및 라벨링
# 6. 출력
'''
anaconda prompt 에서 가상환경 설치 후 -->  conda install -c conda-forge dlib
'''
import cv2, dlib
import numpy as np
from keras.models import load_model
from imutils import face_utils

from keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input


def crop_eye(img, eye_points):
    x1, y1 = np.amin(eye_points, axis=0)
    x2, y2 = np.amax(eye_points, axis=0)
    cx, cy = (x1 + x2) / 2, (y1 + y2) / 2

    w = (x2 - x1) * 1.2
    h = w * IMG_SIZE[1] / IMG_SIZE[0]

    margin_x, margin_y = w / 2, h / 2

    min_x, min_y = int(cx - margin_x), int(cy - margin_y)
    max_x, max_y = int(cx + margin_x), int(cy + margin_y)

    eye_rect = np.rint([min_x, min_y, max_x, max_y]).astype(np.int)

    eye_img = gray[eye_rect[1]:eye_rect[3], eye_rect[0]:eye_rect[2]]

    return eye_img, eye_rect



IMG_SIZE = (34, 26)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('cascadeclassifier/shape_predictor_68_face_landmarks.dat')

# Face,eyes detection XML load and trained model loading
#face_detection = cv2.CascadeClassifier('cascadeclassifier/haarcascade_frontalface_default.xml')
#eye_detection = cv2.CascadeClassifier('cascadeclassifier/haarcascade_eye.xml')






#모델 불러오기
eye_model = load_model('models/2021_02_03_15_05_07.h5')
#eye_model.summary()

#웹캠 열기
try:
    camera = cv2.VideoCapture(0)
except:
    print("camera loading error\n")

    
while camera.isOpened(): #카메라 on
    
    # read frame from webcam
    ret, img_ori = camera.read()
    
    if not ret:
        break
    
    
    img_ori = cv2.resize(img_ori, dsize=(0, 0), fx=0.5, fy=0.5)

    img = img_ori.copy()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = detector(gray)

    for face in faces:
        shapes = predictor(gray, face)
        shapes = face_utils.shape_to_np(shapes)

        eye_img_l, eye_rect_l = crop_eye(gray, eye_points=shapes[36:42])
        eye_img_r, eye_rect_r = crop_eye(gray, eye_points=shapes[42:48])

        eye_img_l = cv2.resize(eye_img_l, dsize=IMG_SIZE)
        eye_img_r = cv2.resize(eye_img_r, dsize=IMG_SIZE)
        eye_img_r = cv2.flip(eye_img_r, flipCode=1)

        #창 2개 따로 생성
        cv2.imshow('l', eye_img_l) 
        cv2.imshow('r', eye_img_r)

        eye_input_l = eye_img_l.copy().reshape((1, IMG_SIZE[1], IMG_SIZE[0], 1)).astype(np.float32) / 255.
        eye_input_r = eye_img_r.copy().reshape((1, IMG_SIZE[1], IMG_SIZE[0], 1)).astype(np.float32) / 255.

        pred_l = eye_model.predict(eye_input_l)
        pred_r = eye_model.predict(eye_input_r)
        
        #print(pred_l)

        # visualize
        state_l = 'O %.1f' if pred_l > 0.1 else '- %.1f'
        state_r = 'O %.1f' if pred_r > 0.1 else '- %.1f'

        state_l = state_l % pred_l
        state_r = state_r % pred_r

        cv2.rectangle(img, pt1=tuple(eye_rect_l[0:2]), pt2=tuple(eye_rect_l[2:4]), color=(255,255,255), thickness=2)
        cv2.rectangle(img, pt1=tuple(eye_rect_r[0:2]), pt2=tuple(eye_rect_r[2:4]), color=(255,255,255), thickness=2)

        cv2.putText(img, state_l, tuple(eye_rect_l[0:2]), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
        cv2.putText(img, state_r, tuple(eye_rect_r[0:2]), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
    
    cv2.imshow('result', img)
    
 


    # q to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# release resources   
camera.release()
cv2.destroyAllWindows()
    

<ipython-input-3-93cd47710814>:33: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eye_rect = np.rint([min_x, min_y, max_x, max_y]).astype(np.int)


[[0.87783456]]
[[0.9392106]]
[[0.68043333]]
[[0.771839]]
[[0.68989617]]
[[0.63136834]]
[[0.68756104]]
[[0.34842268]]
[[0.37965304]]
[[0.3473497]]
[[0.49545375]]
[[0.32930604]]
[[0.28600296]]
[[0.82012]]
[[0.80351317]]
[[0.55621874]]
[[0.7376966]]
[[0.7789035]]
[[0.6703783]]
[[0.40927356]]
[[0.5698042]]
[[0.3605757]]
[[0.5542224]]
[[0.13044414]]
[[0.21022063]]
[[0.16431269]]
[[0.2727813]]
[[0.15002963]]
[[0.4703032]]
[[0.5768209]]
[[0.6473392]]
[[0.67632306]]
[[0.56275487]]
[[0.6204139]]
[[0.4177953]]
[[0.75596654]]
[[0.746976]]
[[0.8098114]]
[[0.5342509]]
[[0.59270644]]
[[0.53715193]]
[[0.76463175]]
[[0.72889054]]
[[0.82871914]]
[[0.6750848]]
[[0.6993667]]
[[0.03805441]]
[[0.01453099]]
[[0.6845877]]
[[0.8851383]]
[[0.7381931]]
[[0.9053663]]
[[0.8052785]]
[[0.8864622]]
[[0.76027685]]
[[0.88188195]]
[[0.9017212]]
[[0.83628273]]
[[0.00909629]]
[[0.02018419]]
[[0.008771]]
[[0.01277682]]
[[0.01307553]]
[[0.01958689]]
[[0.34125155]]
[[0.4696759]]
[[0.01796049]]
[[0.25160775]]
[[0.47888634]]
